In [2]:
def bwt(text):
    rotations = []
    for i in range(len(text)):
        rotation = text[len(text)-i:] + text[0:len(text)-i]
        rotations.append(rotation)
    rotations = sorted(rotations)
    return "".join([s[-1] for s in rotations])

In [3]:
def ibwt(text):
    inverted = ""
    
    symbol2index = [(symbol,i) for i, symbol in enumerate(text)]
    sorted_text = sorted(symbol2index)
    
    next_symbol = "$"
    next_index = text.index(next_symbol)
    while len(inverted)<len(text):
        next_symbol, next_index = sorted_text[next_index]
        inverted += next_symbol
            
    return inverted

In [3]:
def bwtmatch(text, patterns):
    symbol2index = [(symbol,i) for i, symbol in enumerate(text)]
    sorted_text = sorted(symbol2index)
    first2last = {k:v for k, v in zip(sorted_text, symbol2index)}
    
    pattern_frequencies = []
    for pattern in patterns:
        rpattern = pattern[::-1]
    
        rprefixes = [v for k, v in first2last.items() if v[0]==rpattern[0]]
        for c in rpattern[1:]:
            newprefixes = []
            for rprefix in rprefixes:
                t = first2last[rprefix]
                if t[0] == c:
                    newprefixes.append(t)
            rprefixes = newprefixes
        
        pattern_frequencies.append(len(rprefixes))
        
    return pattern_frequencies

In [4]:
bwt_text = "TCCTCTATGAGATCCTATTCTATGAAACCTTCA$GACCAAAATTCTCCGGC"
patterns = "CCT CAC GAG CAG ATC".split(" ")

In [5]:
bwtmatch(bwt_text, patterns)

[2, 1, 1, 0, 1]

In [13]:
with open("../data/dataset_300_8.txt", "r") as fin:
    lines = []
    for line in fin:
        lines.append(line.strip())
    patterns = lines[1].split(" ")
    print(" ".join([str(i) for i in bwtmatch(lines[0], patterns)]))

0 0 1 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 0 1 0 1 1 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 1 0 0 0 1 1 0 1 1 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 1 1 0 1 1 0 0 1 2 1 1 1 0 0 1 1 0 0 0 1 1 0 0 1 0 1 0 0 1 1 1 1 0 1 1 0 1 0 1 1 0 0 0 0 1 1 1 0 0 1 0 1 0 0 1 1 0 1 0 0 1 0 0 0 1 0 0 1 0 0 1 1 1 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 0 1 1 0 0 1 0 1 1 0 1 1 1 0 1 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 0 1 1 0 1 1 1 1 1 1 0 0 1 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 1 1 0 0 1 0 1 0 0 0 1 0 0 0 1 1 1 1 1 0 0 0 1 1 1 0 1 0 0 1 1 0 1 0 0 0 0 0 0 1 1 0 1 1 0 1 1 1 0 0 1 0 0 1 1 1 0 1 1 0 1 1 0 1 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 1 1 0 1 0 0 1 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 1 0 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 0 1 1 0 1 0 1 0 1 0 1 1 1 0 0 0 0 1 1 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 0 0 1 0 1 1 

(asizeof) Memory used by dictionary per pair: 93.0872
(asizeof) Memory used by list of tuples per pair: 136.3488
(flatsize) Memory used by dictionary per pair: 29.5008
(flatsize) Memory used by list of tuples per pair: 8.7624
(sof) Memory used by dictionary per pair: 85.5
(sof) Memory used by list of tuples per pair: 128.7616


In [13]:
from __future__ import print_function
from sys import getsizeof, stderr
from itertools import chain
from collections import deque
try:
    from reprlib import repr
except ImportError:
    pass

def total_size(o, handlers={}, verbose=False):
    """ Returns the approximate memory footprint an object and all of its contents.

    Automatically finds the contents of the following builtin containers and
    their subclasses:  tuple, list, deque, dict, set and frozenset.
    To search other containers, add handlers to iterate over their contents:

        handlers = {SomeContainerClass: iter,
                    OtherContainerClass: OtherContainerClass.get_elements}

    """
    dict_handler = lambda d: chain.from_iterable(d.items())
    all_handlers = {tuple: iter,
                    list: iter,
                    deque: iter,
                    dict: dict_handler,
                    set: iter,
                    frozenset: iter,
                   }
    all_handlers.update(handlers)     # user handlers take precedence
    seen = set()                      # track which object id's have already been seen
    default_size = getsizeof(0)       # estimate sizeof object without __sizeof__

    def sizeof(o):
        if id(o) in seen:       # do not double count the same object
            return 0
        seen.add(id(o))
        s = getsizeof(o, default_size)

        if verbose:
            print(s, type(o), repr(o), file=stderr)

        for typ, handler in all_handlers.items():
            if isinstance(o, typ):
                s += sum(map(sizeof, handler(o)))
                break
        return s

    return sizeof(o)

In [1]:
#quiz

In [4]:
ibwt("TTCCATTGGA$")

'TGTACCATGT$'